In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('lin_reg').getOrCreate()

In [3]:
from pyspark.ml.regression import LinearRegression

In [4]:
training = spark.read.format('libsvm').load('/FileStore/tables/sample_linear_regression_data.txt')

In [5]:
training.show()

+-------------------+--------------------+
 label| features|
+-------------------+--------------------+
 -9.490009878824548|(10,[0,1,2,3,4,5,...|
 0.2577820163584905|(10,[0,1,2,3,4,5,...|
 -4.438869807456516|(10,[0,1,2,3,4,5,...|
-19.782762789614537|(10,[0,1,2,3,4,5,...|
 -7.966593841555266|(10,[0,1,2,3,4,5,...|
 -7.896274316726144|(10,[0,1,2,3,4,5,...|
 -8.464803554195287|(10,[0,1,2,3,4,5,...|
 2.1214592666251364|(10,[0,1,2,3,4,5,...|
 1.0720117616524107|(10,[0,1,2,3,4,5,...|
-13.772441561702871|(10,[0,1,2,3,4,5,...|
 -5.082010756207233|(10,[0,1,2,3,4,5,...|
 7.887786536531237|(10,[0,1,2,3,4,5,...|
 14.323146365332388|(10,[0,1,2,3,4,5,...|
-20.057482615789212|(10,[0,1,2,3,4,5,...|
-0.8995693247765151|(10,[0,1,2,3,4,5,...|
 -19.16829262296376|(10,[0,1,2,3,4,5,...|
 5.601801561245534|(10,[0,1,2,3,4,5,...|
-3.2256352187273354|(10,[0,1,2,3,4,5,...|
 1.5299675726687754|(10,[0,1,2,3,4,5,...|
 -0.250102447941961|(10,[0,1,2,3,4,5,...|
+-------------------+--------------------+
only showing top 20 rows

In [6]:
lr = LinearRegression(featuresCol='features', labelCol='label', predictionCol='prediction')

In [7]:
lrModel = lr.fit(training)

In [8]:
lrModel.coefficients

Out[11]: DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [9]:
lrModel.intercept

Out[12]: 0.14228558260358093

In [10]:
training_summary = lrModel.summary

In [11]:
# variance of the model
training_summary.r2

Out[14]: 0.027839179518600154

In [12]:
training_summary.rootMeanSquaredError

Out[15]: 10.16309157133015

In [13]:
all_data = spark.read.format('libsvm').load('/FileStore/tables/sample_linear_regression_data.txt')

In [14]:
# First part of split will have 70% of data, the next part will have 30% of data
train_data, test_data = all_data.randomSplit([0.7, 0.3])

In [15]:
train_data.describe().show()

+-------+-------------------+
summary| label|
+-------+-------------------+
 count| 346|
 mean|0.39496168581035407|
 stddev| 10.452309430640598|
 min|-28.571478869743427|
 max| 27.78383192005107|
+-------+-------------------+

In [16]:
test_data.describe().show()

+-------+--------------------+
summary| label|
+-------+--------------------+
 count| 155|
 mean|-0.05132544110123992|
 stddev| 10.0377625874035|
 min| -26.805483428483072|
 max| 26.903524792043335|
+-------+--------------------+

In [17]:
correct_model = lr.fit(train_data)

In [18]:
test_results = correct_model.evaluate(test_data)

In [19]:
test_results.residuals.show()

+-------------------+
 residuals|
+-------------------+
-27.971819389026084|
 -21.87164747235645|
-27.033504997766762|
 -21.33573610520711|
 -20.35863744371532|
-22.321683954799518|
-18.225282322983123|
 -18.22058294479868|
-13.848034242951515|
-12.987371229394554|
-13.250792257891401|
 -16.03175347458452|
-16.719443367969212|
 -18.21688100979951|
-11.307716882605622|
 -14.01418987181616|
-14.532076327791343|
-12.088052704055649|
-14.795640900048202|
-10.243208478633926|
+-------------------+
only showing top 20 rows

In [20]:
test_results.rootMeanSquaredError

Out[27]: 10.30654767228647

In [21]:
unlabeled_data = test_data.select('features')
unlabeled_data.show()

+--------------------+
 features|
+--------------------+
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows

In [22]:
predictions = correct_model.transform(unlabeled_data)
predictions.show()

+--------------------+-------------------+
 features| prediction|
+--------------------+-------------------+
(10,[0,1,2,3,4,5,...| 1.166335960543011|
(10,[0,1,2,3,4,5,...|-1.6392366179665205|
(10,[0,1,2,3,4,5,...| 4.08367906157069|
(10,[0,1,2,3,4,5,...| 1.278253489417899|
(10,[0,1,2,3,4,5,...| 0.9563014135007667|
(10,[0,1,2,3,4,5,...| 3.475761481900934|
(10,[0,1,2,3,4,5,...| 1.9638520475303909|
(10,[0,1,2,3,4,5,...| 2.4398979121753777|
(10,[0,1,2,3,4,5,...|-1.5893505504797028|
(10,[0,1,2,3,4,5,...|-2.3614999259846985|
(10,[0,1,2,3,4,5,...|-2.0601883315248886|
(10,[0,1,2,3,4,5,...| 1.2096005648333288|
(10,[0,1,2,3,4,5,...| 2.3904648588937687|
(10,[0,1,2,3,4,5,...| 4.444439448096639|
(10,[0,1,2,3,4,5,...| -2.112877893285136|
(10,[0,1,2,3,4,5,...| 1.4556140829599729|
(10,[0,1,2,3,4,5,...| 2.0313025424362876|
(10,[0,1,2,3,4,5,...| 0.1830658013805352|
(10,[0,1,2,3,4,5,...| 2.9174734000813602|
(10,[0,1,2,3,4,5,...|-1.6141418867954997|
+--------------------+-------------------+
only showing top 20 rows

In [23]:
final_data.describe().show()

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-1857495520278009> in <module> 
 ----> 1 final_data . describe ( ) . show ( ) 

 NameError : name 'final_data' is not defined